In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import mtalg
from financepy.utils.math import N, n_vect

####################################################################
# FINANCEPY BETA Version 0.310 - This build:  25 Aug 2023 at 16:17 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Задача 1 ##

### Постановка задачи
$
\text{Даны X и Y - cлучайные величины с нормальным распределением: } X \sim \mathcal{N}(e_1, q_1^2) \text{ и } Y \sim \mathcal{N}(e_2, q_2^2) \text{, где corr}(X, Y) = p.
$
$
\text{Необходимо найти такие константы a и b, минимизирующие } \text{Var}[S] \text{, где } S = aX + bY \text{, такие что } a + b = 1, 0 \leq a \leq 1, 0 \leq b \leq 1
$

### Аналитическое решение
$\text{По условию, } \text{cov}(X,Y) = p q_1 q_2 \\$
$\text{Воспользуемся свойствами дисперсии и сформулируем задачу формально: } \\ \text{Min } \text{Var}[S] = a^2 q_1^2 + b^2 q_2^2 + 2abpq_1q_2 \\ \text{при ограничениях: } a + b = 1, 0 \leq a \leq 1, 0 \leq b \leq 1 \\$

$\text{Это задача квадратичного программирования c ограничениями. }$
$\text{В общем случае, такие задачи не имеют аналитического решения.}$ $\text{Попробуем исследовать частное решение, применив метод множителей Лагранжа: } \\ $

$  \text{Создадим функцию Лангража: }L(a, b, \lambda) = a^2 q_1^2 + b^2 q_2^2 + 2abpq_1q_2 + \lambda(1 - a - b)$

$ \text{Найдем частные производные для } a, b, \lambda\ \text{ и приравняем их к нулю:} \\$
$ \frac{\partial L}{\partial a} = 2aq_1^2 + 2bpq_1q_2 - \lambda = 0 \\ $
$ \frac{\partial L}{\partial b} = 2bq_2^2 + 2apq_1q_2 - \lambda = 0 \\ $
$ \frac{\partial L}{\partial \lambda} = 1 - a - b = 0 $
$\text{} \\$
$\text{Решив эту систему уравнений относительно a и b,}$
$\text{мы получим аналитическое решение. Выразим a и b через } \lambda \text{:}\\$
$ a = \frac{\lambda - 2bpq_1q_2}{2q_1^2} \\$
$ b = \frac{\lambda - 2apq_1q_2}{2q_2^2} \\$
$\text{Таким образом, параметры a и b не выражаются в явном виде через }\lambda$
$\text{и демонстрируют сложную нелинейную зависимость.}$ 
$\text{Дальнейшее аналитическое исследование задачи не целесообразно.}$




### Численное решение

In [2]:
q1= 1
q2= 2
p = -0.25

# Целевая функция
def objective(x):
    a, b = x
    return a**2 * q1**2 + b**2 * q2**2 + 2 * a * b * p * q1 * q2

# Ограничения
constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1},
               {'type': 'ineq', 'fun': lambda x: x[0]},
               {'type': 'ineq', 'fun': lambda x: x[1]})

x0 = [0.5, 0.5]
result = minimize(objective, x0, constraints=constraints)
opt_a, opt_b = result.x
print(f'_w1: {round(opt_a,4)}, _w2: {round(opt_b,4)}, _minVar[S]: {result.fun}')


_w1: 0.75, _w2: 0.25, _minVar[S]: 0.625


## Задача 2 ##

Напишите алгоритм монте карло для симуляции цены
Используйте сгенерированные цены для нумерической оценки опционов используя Delta Hedging на каждом шаге цены (игнорируя комиссию)

Параметры Опциона Для оценки :
Strike = 1000
Implied Volatility = 0.3
Time to Expiration = Half a Year
interest rate = 0

Параметры для Монте Карло
Start Spot price = 1000
Realised Volatility = 0.2
Number of Paths = 50000
Number of Steps = N days untill the expiration

Напишите алгоритм нумерической апроксимации PNL данного опциона на момент экспирации. 
Финальное решение включающее в себя генерирование всех путей и оценку PNLa должно находиться в одном cell-e. На вывод должен быть принт финального пнла и тайминг экзекьюшена всего решения.

In [9]:
%%timeit
strike = 1000
ImpVol = 0.3
expiry = 0.5
rate = 0

spot = 1000
RealVol = 0.2
n_paths = 50000

def gbm(mu=0, rv=1, sims= 50000, T= 0.5, S0= 1000, cal=360):
    sims= int(sims)
    steps= int(cal * T)
    dt= T/steps
    St= np.exp(
        (mu - rv**2/2)*dt + rv * mtalg.random.normal(0, np.sqrt(dt), size=(sims, steps-1)).T
        )

    St = np.vstack([np.ones(sims), St])
    St = S0 * St.T.cumprod(axis=1)
    return St.T

def deltaGK(opt, S, K, iv, tau, sims, rf=0, rd=0):
    tau=np.linspace(start= 0, stop= tau, num=sims+1)
    tau= np.tile(tau[::-1][0:sims], reps= S.shape[0]).reshape(S.shape[0], sims)
    d1 = (np.log(S/K) + (rd - rf + (iv**2)/2)*tau)/(iv*np.sqrt(tau))
    d1= d1.reshape(S.shape[0], sims)
    delta= n_vect(d1)
    print(delta)
        
    if opt == 'call':
        deltaCall= delta
        return deltaCall
    elif opt == 'put':
        deltaPut= delta - 1
        return deltaPut

def deltaHedge(cal=360):
    St= gbm(mu=0, rv=RealVol, sims= n_paths, T= expiry, S0= spot, cal=cal)

    deltas= deltaGK('call', S=St, K=strike, iv= ImpVol, sims=n_paths, \
        tau=expiry, rd=rate)

    dh= np.diff(deltas.T).T
    print(dh)
    dh= np.vstack([deltas[0], dh])
    pnl_apx= np.sum((dh*St).T, axis=1)

    print(f'PnL Approximation: {round(np.mean(pnl_apx),2)} ± {round(np.std(pnl_apx),2)}')



deltaHedge()

[[5.42235068e-01 5.42234648e-01 5.42234227e-01 ... 5.00327767e-01
  5.00267621e-01 5.00189237e-01]
 [5.54076616e-01 5.60530902e-01 5.55235923e-01 ... 9.99999957e-01
  1.00000000e+00 0.00000000e+00]
 [5.38518156e-01 5.53910496e-01 5.67280622e-01 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 ...
 [6.35093967e-01 2.45373782e-01 8.07979958e-01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.37161339e-01 2.57581244e-01 8.17699211e-01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.16334163e-01 2.35729987e-01 8.10884127e-01 ... 1.00000000e+00
  4.03677092e-13 0.00000000e+00]]
[[ 1.18415472e-02  1.82962539e-02  1.30016967e-02 ...  4.99672190e-01
   4.99732379e-01 -5.00189237e-01]
 [-1.55584599e-02 -6.62040532e-03  1.20446986e-02 ...  4.30241621e-08
  -1.11022302e-16  0.00000000e+00]
 [-7.03973529e-03 -5.50487731e-03 -2.46549346e-02 ...  0.00000000e+00
   1.11022302e-16  1.92674188e-06]
 ...
 [ 1.56969344e-02  1.27595812e-02 -1.39107450e-02 ...  0.00000000e+00
   0.00000

## Задача 3 ##

Даны случайные величины с нормальным распределением : X~N(mu,s), mu= 0 , s - ?
случайные величины эквивалентны дневному приросту цены.
Среднее абсолютное отклонение (Mean Average Deviation) (X) = 1%
чему равен s в годовом эквиваленте если мы закладываем 252 дня в году?


In [4]:
def extractAnnualVar(mad=0.01, cal=252):
    centeredSumX= mad*cal
    dailyVar= centeredSumX**2/(cal-1)
    annualisedVar= dailyVar*cal
    return annualisedVar

print(extractAnnualVar())

6.375700398406376
